In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3 
http = urllib3.PoolManager()
import warnings
warnings.filterwarnings('ignore')

In [2]:
from stocksymbol import StockSymbol
#Gets all the stock names from this API
api_key = 'bcfba455-103b-4a95-9c1b-c8f6877c3a6d'
ss = StockSymbol(api_key)
markets = ['us','ar','au','at','be','br','ca','cn','cz','dk','eg','fi','fr','de','gr','hk','hu','is','in','id','ie','il','it','jp','kr','lv','lt','my','mx','nl','nz','no','pl','pt','qa','ru','sa','sg','za','es','lk','se','ch','tw','th','tr','gb','ve']
lst_with_stocks = []
for i in markets:
    stock_list = ss.get_symbol_list(market = i, symbols_only=True)
    lst_with_stocks = lst_with_stocks + stock_list

In [3]:
#list of the rows on the website I'm scappring the information from
gray_list = ['COGS','Gross Profit Ratio', 'Operating Expenses','R&D Expenses','Selling, G&A Exp.','General and Admin. Exp.','Selling and Marketing Exp','Other Expenses', 'COGS and Expenses', 'Interest Income', 'Interest Expense', 'Depreciation and Amortization', 'EBITDA ratio', 'Operating Income ratio', 'Total Other Income Exp.(Gains)', 'Income Before Tax ratio', 'Income Tax Expense (Gain)', 'Net Income Ratio', 'EPS', 'EPS Diluted', 'Weighted Avg. Shares Outs.', 'I_SEC Link', 'Cash and Cash Equivalents', 'Short-Term Investments', 'Cash & Short-Term Investments', 'Net Receivables', 'Inventory', 'Other Current Assets', 'PP&E', 'Goodwill', 'Intangible Assets', 'Goodwill and Intangible Assets', 'Investments', 'Tax Assets', 'Other Non-Current Assets','Total Non-Current Assets', 'Other Assets', 'Accounts Payable', 'Short-Term Debt', 'Tax Payable', 'Deferred Revenue', 'Other Current Liabilities', 'Long-Term Debt', 'Deferred Revenue', 'Deferred Tax Liabilities', 'Other Non-Current Liabilities', 'Total Non-Current Liabilities', 'Other Liabilities', 'Capital Lease Obligations', 'Preferred Stock', 'Common Stock', 'Retained Earnings', 'Other Compreh. Income(Loss)', 'Other Total Stockhold. Equity','Total Liab.&Stockhold. Equity', 'Minority Interest', 'Total Liabilities & Equity','B_SEC Link', 'Net_Income', 'Depreciation_and_Amortization', 'Deferred_Income_Tax', 'Stock Based Compensation', 'Change in Working Capital', 'Accounts Receivable', 'Inventory', 'Accounts Payable', 'Other Working Capital', 'Other Non-Cash Items', 'CAPEX', 'Acquisitions Net', 'Purchases of Investments', 'Sales/Maturities of Investments', 'Other Investing Activities', 'Debt Repayment', 'Common Stock Issued', 'Common Stock Repurchased', 'Dividends Paid', 'Other Financing Activities', 'Effect of Forex Changes on Cash', 'Cash at the End of Period', 'Cash at the Beginning of Period', 'C_SEC Link', 'Dunno']

In [5]:
def go_to_site(stock:str, site = 'https://roic.ai/financials/'):
    url = site + stock 
    resp = http.request('GET', url)
    soup = BeautifulSoup(resp.data, 'html.parser')
    return soup

In [6]:
def get_table(soup, text, division = 'div', parameter = 'class', ):
    mydivs = soup.find_all(division, {parameter: text })#"w-[80px] select-none pr-1 grow text-right text-xs 2xl:text-sm font-medium"})
    lst = []
    for i in mydivs:
        lst.append(i.get_text())
    return lst


In [7]:
def from_txt_to_number(lst):
    #Transforms the txt (percentages, missing numbers, negatives numbers) into numbers
    new_lst = []
    for j in lst:
        number = ''
        if ',' in j:
            j = j.replace(',','')
        if '%' in j:
            j = j.replace('%','')
        if '(' in j:
            j = j.replace('(','-')
            j = j.replace(')','')
        if j == '- -' or j== 'Link':
            number = None
        if j == 'TTM':
            number = 2022
        else:
            try:
                number = float(j)
            except:
                number = None
        new_lst.append(number)
    return new_lst



In [8]:
def get_stock_table(stock, export = True):
    soup = go_to_site(stock)
    lst = get_table(soup, text = 'w-[80px] select-none pr-1 grow text-right text-xs 2xl:text-sm font-medium', division = 'div', parameter = 'class' )
    #if the website doesn't have the information, it goes to the next stock
    if lst == []:
        return []
    final = from_txt_to_number(lst)
    number_of_years = get_table(soup, text = 'w-[80px] select-none bg-white pr-1 grow font-light text-right text-sm text-neutral-400', division = 'div', parameter = 'class' )
    years = from_txt_to_number(number_of_years)
    get_columns = get_table(soup, text = 'text-left select-none break-words pl-2 truncate font-medium', division = 'span', parameter = 'class' )
    df1 = pd.DataFrame(np.transpose(np.reshape(final,(len(get_columns),len(years)))), index = years, columns = get_columns)
    
    #Now get the numbers in gray present on the financials table
    
    gray_numbers = get_table(soup, text = 'w-[80px] select-none pr-1 grow text-right text-xs 2xl:text-sm font-light', division = 'div', parameter = 'class' )
    gray_numbers = from_txt_to_number(gray_numbers)
    get_columns = get_table(soup, text = 'text-left select-none break-words pl-2 truncate font-medium', division = 'span', parameter = 'class' )
    df2 = pd.DataFrame(np.transpose(np.reshape(gray_numbers,(len(gray_list),len(years)))), index = years, columns = gray_list)
    
    # Now goes to the summary page for the site and gets that information as well

    soup = go_to_site(stock, site = 'https://roic.ai/company/')
    summary_numbers = get_table(soup, text = 'min-w-[50px] w-full grow text-center text-xs 2xl:text-sm h-full flex items-center justify-center', division = 'div', parameter = 'class' )
    summary_numbers = from_txt_to_number(summary_numbers)
    number_of_years_for_summary = get_table(soup, text = 'min-w-[50px] w-full grow text-center text-sm font-light text-neutral-400 h-full flex items-center justify-center', division = 'div', parameter = 'class' )
    number_of_years_for_summary = from_txt_to_number(number_of_years_for_summary)
    get_columns_summary = get_table(soup, text = 'text-left break-words pl-2 font-light truncate', division = 'span', parameter = 'class' )
    df3 = pd.DataFrame(np.transpose(np.reshape(summary_numbers,(len(get_columns_summary),len(number_of_years_for_summary)))), index = number_of_years_for_summary, columns = get_columns_summary)
    df = df1.join(df2)
    df = df.join(df3)
    if export:
        df.to_csv('csvs/' + stock + '.csv', encoding = 'utf-8-sig', sep = ';')
    return df

In [9]:
def get_all_stocks():
#executes everything
    no = 0
    yes = 0
    for i in lst_with_stocks:
        print(i)
        try:
            final_stock = get_stock_table(i)
            if len(final_stock) != 0:
                yes +=1
            else:
                no +=1
        except:
            no +=1
            pass
    print('We are done!')
    print("We didn't get information for " + str(no) + ' stocks.')
    print("We were able to get the information for " + str(yes) + ' stocks.')

        

In [10]:
final = get_all_stocks()

HSY.MU
HSY.F
HSY.DE
PPQ.DU
PPQ.MU
PPQ.BE
PPQ.DE
PPQ.SG
PPQ.F
TGR.HA
TGR.DU
TGR.MU
TGR.SG
TGR.HM
TGR.BE
TGR.F
TGR.DE
N7T.DE
N7T.DU
N7T.MU
N7T.BE
N7T.SG
N7T.F
MTS1.F
MTS1.SG
MTS1.DU
MTS1.MU
MTS1.BE
2KD.MU
2KD.DE
2KD.DU
2KD.F
2KD.BE
2KD.SG
S6M.BE
S6M.SG
S6M.F
S6M.DU
S6M.MU
NEF.DU
NEF.F
NEF.SG
NEF.DE
NEF.BE
NEF.MU
CGM.HA
CGM.DU
CGM.F
CGM.MU
CGM.SG
CGM.BE
CGM.DE
CGM.HM
ADM.BE
ADM.MU
ADM.HM
ADM.SG
ADM.DU
ADM.HA
ADM.DE
ADM.F
GOB.BE
GOB.MU
GOB.DU
GOB.DE
GOB.F
GOB.HM
GOB.HA
GOB.SG
SRT3.HA
SRT.HM
SRT.F
SRT.BE
SRT.DU
SRT3.MU
SRT3.DU
SRT3.BE
SRT.SG
SRT.MU
SRT.HA
SRT.DE
SRT3.HM
SRT3.DE
SRT3.SG
SRT3.F
RWL.HA
RWL.F
RWL.DE
RWL.MU
RWL.DU
RWL.SG
RWL.HM
RWL.BE
TT8.SG
TT8.F
TT8.DU
TT8.BE
TT8.MU
TT8.DE
RYS1.DU
RYS1.BE
RYS1.SG
RYS1.F
RYS1.HM
RYS1.MU
RYS1.HA
RYS1.DE
ERT.HM
ERT.BE
ERT.SG
ERT.HA
ERT.DE
ERT.F
ERT.DU
ERT.MU
NRN.DU
NRN.BE
NRN.DE
NRN.F
NRN.SG
NRN.MU
AS7.F
AS7.DU
AS7.BE
AS7.SG
AS7.MU
ZYA.SG
ZYA.MU
ZYA.DU
ZYA.F
ZYA.DE
ZYA.HA
RSO.DE
RSO.SG
RSO.F
RSO.BE
RSO.DU
RSO.MU
RSO.HM
RSO.HA
E2F.DU
E2F.BE
E2F.DE